In [3]:
import grid2op
from grid2op.utils import ScoreL2RPN2022
from agent_set import make_agent
import numpy as np
import os
import json
from lightsim2grid import LightSimBackend

In [4]:
env_names = ["input_data_val"]
env_scores = {}

for env_name in env_names:
  env = grid2op.make(env_name, backend=LightSimBackend())
  with open(os.path.join(env._init_env_path, "scenarios_info_seeds.json")) as f:
    seeds = json.load(f)

  env_seeds = [int(seeds[os.path.split(el)[-1]]["seed"]) for el in sorted(env.chronics_handler.real_data.subpaths)]
  scorer = ScoreL2RPN2022(env, env_seeds, nb_process_stats=1, nb_scenario=len(env_seeds), verbose=2)
  agent = make_agent(env, ".", "saved_model")
  scores, _, _ = scorer.get(agent)
  print(f"Score: {np.mean(scores)}")
  env_scores[env_name] = scores

Starts the evaluation of the agent


episode: 100%|██████████| 52/52 [06:42<00:00,  7.73s/it]


Start the evaluation of the scores
Score: 23.57823946132345


In [5]:
for k, v in env_scores.items():
  print(k, np.mean(v))

input_data_val 23.57823946132345


In [6]:
np.save("study_set_agents/scores_set_agents", env_scores["input_data_val"])